# **MVP – Engenharia de Dados**
## **Análise Logística e Satisfação no E-commerce Brasileiro**

**Disciplina:** Sprint – Engenharia de Dados  
**Autor:** Eduardo Belisario Nicolau  
**Dataset:** Brazilian E-Commerce Public Dataset by Olist  
**Fonte:** Kaggle  
**Período:** 2016–2018  

Este trabalho apresenta a construção de um **pipeline de dados em nuvem**, desde a coleta até a análise, utilizando dados públicos de e-commerce, com foco em **desempenho logístico e satisfação do cliente**.


## 🎯 **Objetivo**

O objetivo deste MVP é **construir um pipeline de dados em nuvem**, utilizando o **Google Colab** como ambiente de execução, capaz de **coletar, organizar, modelar, transformar e analisar dados reais de e-commerce**, com foco em **desempenho logístico e satisfação do cliente**.

Busca-se compreender como **fatores logísticos, geográficos e operacionais** influenciam:
- O tempo de entrega dos pedidos  
- Os custos de frete  
- A avaliação dos clientes  

Embora a disciplina direcione o suporte à plataforma **Databricks**, optou-se pelo **Google Colab** como ambiente de execução em nuvem, o qual **atende integralmente aos requisitos do MVP**, conforme permitido no enunciado da disciplina.


## ❓ **Perguntas de Negócio**

- Qual é o **tempo médio de entrega** dos pedidos?
- Quais **estados** apresentam maiores variações no desempenho logístico?
- Existe relação entre **atraso na entrega** e **avaliação do cliente**?
- Quais **categorias de produto** possuem maior tempo médio de entrega?
- O **custo de frete** varia significativamente por região?
- Há diferenças no comportamento de compra entre **grandes centros urbanos** e **cidades menores**?


## 🏗️ **Arquitetura do Pipeline**

O pipeline foi estruturado em **três camadas lógicas**, seguindo boas práticas de Engenharia de Dados:

- **Bronze:** dados brutos coletados diretamente do Kaggle  
- **Silver:** dados tratados, padronizados e enriquecidos  
- **Gold:** dados integrados e modelados em **Esquema Estrela** para análise  

O **Google Colab** foi utilizado como ambiente de execução em nuvem, simulando um pipeline analítico completo com **ETL explícito**.


In [14]:
# Coleta dos dados utilizando kagglehub (reprodutível e automatizada)

import kagglehub
import os
import pandas as pd

DATASET_PATH = kagglehub.dataset_download("olistbr/brazilian-ecommerce")

print("Dataset disponível em:", DATASET_PATH)
os.listdir(DATASET_PATH)


Using Colab cache for faster access to the 'brazilian-ecommerce' dataset.
Dataset disponível em: /kaggle/input/brazilian-ecommerce


['olist_customers_dataset.csv',
 'olist_sellers_dataset.csv',
 'olist_order_reviews_dataset.csv',
 'olist_order_items_dataset.csv',
 'olist_products_dataset.csv',
 'olist_geolocation_dataset.csv',
 'product_category_name_translation.csv',
 'olist_orders_dataset.csv',
 'olist_order_payments_dataset.csv']

## 📥 **Coleta dos Dados**

Os dados foram coletados diretamente do **Kaggle** utilizando a biblioteca **kagglehub**, que permite o download **programático e reprodutível** de datasets públicos.

Essa abordagem:
- Elimina a necessidade de downloads manuais  
- Garante **reprodutibilidade total**  
- Facilita a execução do pipeline por avaliadores e outros usuários  


In [15]:
# Leitura dos datasets (Camada Bronze)

orders = pd.read_csv(os.path.join(DATASET_PATH, "olist_orders_dataset.csv"))
order_items = pd.read_csv(os.path.join(DATASET_PATH, "olist_order_items_dataset.csv"))
customers = pd.read_csv(os.path.join(DATASET_PATH, "olist_customers_dataset.csv"))
products = pd.read_csv(os.path.join(DATASET_PATH, "olist_products_dataset.csv"))
sellers = pd.read_csv(os.path.join(DATASET_PATH, "olist_sellers_dataset.csv"))
reviews = pd.read_csv(os.path.join(DATASET_PATH, "olist_order_reviews_dataset.csv"))
payments = pd.read_csv(os.path.join(DATASET_PATH, "olist_order_payments_dataset.csv"))


## 🧩 **Modelagem de Dados**

O **grão** definido para a tabela fato é **item de pedido**.

### **Tabela Fato**
- **fact_order_items:** representa cada item vendido em um pedido

### **Dimensões**
- **dim_orders:** informações temporais e status do pedido  
- **dim_customers:** dados geográficos do cliente  
- **dim_products:** atributos do produto  
- **dim_sellers:** informações do vendedor  

O modelo segue o padrão de **Esquema Estrela**, amplamente utilizado em **Data Warehouses** para análises analíticas.


In [16]:
# Camada Silver – Tratamento e enriquecimento

date_cols = [
    "order_purchase_timestamp",
    "order_delivered_customer_date",
    "order_estimated_delivery_date"
]

for col in date_cols:
    orders[col] = pd.to_datetime(orders[col], errors="coerce")

# Métrica de atraso de entrega
orders["delivery_delay_days"] = (
    orders["order_delivered_customer_date"]
    - orders["order_estimated_delivery_date"]
).dt.days


## 🧹 **Camada Silver – Tratamento e Enriquecimento**

Nesta etapa foram realizadas:
- Conversão de colunas de data para o tipo **datetime**
- Criação da métrica **delivery_delay_days**

**Interpretação da métrica:**
- Valores **positivos** indicam atraso logístico  
- Valores **zero ou negativos** indicam entregas no prazo ou antecipadas  


In [17]:
# Camada Gold – Construção da tabela fato integrada

fact = (
    order_items
    .merge(orders, on="order_id", how="left")
    .merge(customers, on="customer_id", how="left")
    .merge(products, on="product_id", how="left")
    .merge(sellers, on="seller_id", how="left")
)


## 📚 **Catálogo de Dados (Resumo)**

### **fact_order_items**
- **order_id:** identificador do pedido  
- **product_id:** identificador do produto  
- **price:** valor do item (R$)  
- **freight_value:** custo do frete (R$)  
- **delivery_delay_days:** atraso da entrega em dias  
  - Domínio esperado: aproximadamente **[-30, +100]**

### **dim_customers**
- **customer_state:** estado (UF)  
- **customer_city:** cidade  

### **dim_products**
- **product_category_name:** categoria do produto  


In [18]:
def quality_report(df):
    return pd.DataFrame({
        "null_pct": df.isnull().mean() * 100,
        "unique_values": df.nunique()
    }).sort_values("null_pct", ascending=False)

quality_report(fact)


,null_pct,unique_values
order_delivered_customer_date,2.178429,95664
delivery_delay_days,2.178429,198
product_category_name,1.422992,73
product_name_lenght,1.422992,66
product_description_lenght,1.422992,2960
product_photos_qty,1.422992,19
order_delivered_carrier_date,1.059920,81017
product_length_cm,0.015979,99
product_height_cm,0.015979,102
product_weight_g,0.015979,2204


## 🔍 **Análise de Qualidade dos Dados**

Observa-se a presença de valores nulos principalmente em colunas relacionadas à **data de entrega**, o que é esperado para pedidos **ainda não entregues ou cancelados**.

Esses valores **não caracterizam erro de qualidade**, mas refletem o **estado operacional** do pedido dentro do processo logístico.


In [19]:
# Diferença média entre data de entrega e data estimada por estado

fact.groupby("customer_state")["delivery_delay_days"].mean().sort_values(ascending=False)


,delivery_delay_days
customer_state,
AL,-8.735363
MA,-9.906250
SE,-10.002667
ES,-10.646292
BA,-10.982623
CE,-11.103787
SP,-11.207911
MS,-11.229346
PI,-11.527725


In [20]:
# Relação entre atraso e avaliação
fact_reviews = fact.merge(reviews, on="order_id", how="left")


In [21]:
fact_reviews.groupby(
    pd.cut(fact_reviews["delivery_delay_days"], [-999, 0, 3, 7, 100]),
    observed=False
)["review_score"].mean()


,review_score
delivery_delay_days,
"(-999, 0]",4.207300
"(0, 3]",3.228858
"(3, 7]",2.088672
"(7, 100]",1.681308


In [22]:
# Categorias com maior tempo médio de entrega
fact.groupby("product_category_name")["delivery_delay_days"].mean().sort_values(ascending=False).head(10)


,delivery_delay_days
product_category_name,
artes_e_artesanato,-6.791667
moveis_colchao_e_estofado,-7.162162
casa_conforto_2,-8.433333
portateis_cozinha_e_preparadores_de_alimentos,-9.500000
pc_gamer,-9.625000
casa_conforto,-9.811189
alimentos,-9.867735
audio,-10.149171
fashion_underwear_e_moda_praia,-10.929134


## ✅ **Conclusões**

Os resultados indicam uma **relação clara entre desempenho logístico e satisfação do cliente**, onde maiores atrasos estão associados a **avaliações mais baixas**.

Observa-se também que fatores **geográficos** influenciam o tempo de entrega, reforçando a logística como elemento estratégico no e-commerce brasileiro.

Esses achados demonstram que a **qualidade logística** impacta diretamente a **experiência do cliente**.


## 📝 **Autoavaliação**

Os objetivos inicialmente propostos foram **majoritariamente atingidos**, especialmente no que se refere à construção de um **pipeline de dados completo** e à análise dos fatores logísticos.

As principais dificuldades encontradas estiveram relacionadas à **padronização de datas** e à **conciliação entre múltiplos conjuntos de dados**.

Como trabalhos futuros, pretendo:
- Automatização do pipeline em uma plataforma dedicada de dados  
- Criação de **dashboards interativos** para acompanhamento contínuo dos indicadores logísticos  
